Current and near-term quantum computers suffer from imperfections, as we repeatedly pointed it out. This is why we cannot run long algorithms, that is, deep circuits on them. A new breed of algorithms started to appear since 2013 that focus on getting an advantage from imperfect quantum computers. The basic idea is extremely simple: run a short sequence of gates where some gates are parametrized. Then read out the result, make adjustments to the parameters on a classical computer, and repeat the calculation with the new parameters on the quantum hardware. This way we create an iterative loop between the quantum and the classical processing units, creating classical-quantum hybrid algorithms.

<img src="figures/hybrid_classical_quantum.svg" alt="Hybrid classical-quantum paradigm" style="width: 400px;"/>

These algorithms are also called variational to reflect the variational approach to changing the parameters. One of the most important example of this approach is the quantum approximate optimization algorithm, which is the subject of this notebook.

# Quantum approximate optimization algorithm

The quantum approximate optimization algorithm (QAOA) is shallow-circuit variational algorithm for gate-model quantum computers that was inspired by quantum annealing. We discretize the adiabatic pathway in some $p$ steps, where $p$ influences precision. Each discrete time step $i$ has two parameters, $\beta_i, \gamma_i$. The classical variational algorithms does an optimization over these parameters based on the observed energy at the end of a run on the quantum hardware.

More formally, we want to discretize the time-dependent $H(t)=(1-t)H_0 + tH_1$ under adiabatic conditions. We achieve this by Trotterizing the unitary. For instance, for time step $t_0$, we can split this unitary as $U(t_0) = U(H_0, \beta_0)U(H_1, \gamma_0)$. We can continue doing this for subsequent time steps, eventually splitting up the evolution to $p$ such chunks:

$$
U = U(H_0, \beta_0)U(H_1, \gamma_0)\ldots U(H_0, \beta_p)U(H_1, \gamma_p).
$$

At the end of optimizing the parameters, this discretized evolution will approximate the adiabatic pathway:

<img src="figures/qaoa_process.svg" alt="Quantum approximate optimization algorithm" style="width: 400px;"/>

The Hamiltonian $H_0$ is often referred to as the driving or mixing Hamiltonian, and $H_1$ as the cost Hamiltonian. The simplest mixing Hamiltonian is $H_0 = -\sum_i \sigma^X_i$, the same as the initial Hamiltonian in quantum annealing. By alternating between the two Hamiltonian, the mixing Hamiltonian drives the state towards and equal superposition, whereas the cost Hamiltonian tries to seek its own ground state.

Let us import the necessary packages first:

In [1]:
import itertools
import numpy as np
from functools import partial, reduce
from qiskit import Aer, QuantumRegister
from qiskit.tools.qi.pauli import Pauli
from qiskit.wrapper import execute as q_execute
from qiskit_aqua import get_initial_state_instance
from qiskit_aqua.operator import Operator
from scipy.optimize import minimize

Now we can define our mixing Hamiltonian on some qubits:

In [2]:
n_qubits = 2

def pauli_z(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[qubit], np.zeros(n_qubits))]])

def pauli_x(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[1, Pauli(np.zeros(n_qubits), eye[qubit])]])

def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[q1], np.zeros(n_qubits)) * Pauli(eye[q2], np.zeros(n_qubits))]])

Hm = reduce(lambda x,y:x+y,
            [pauli_x(i, 1) 
             for i in range(n_qubits)])
Hm.to_matrix()

As an example, we will minimize the Ising problem defined by the cost Hamiltonian $H_c=-\sigma^Z_1 \otimes \sigma^Z_2$.

In [3]:
J = np.array([[0,1],[0,0]])
Hc = reduce(lambda x,y:x+y,
        [product_pauli_z(i,j, -J[i,j])
         for i,j in itertools.product(range(n_qubits), repeat=2)])
Hc.to_matrix()

We set $p=2$ and initialize the $\beta_i$ and $\gamma_i$ parameters:

In [4]:
n_iter = 10 # number of iterations of the optimization procedure
p = 2
beta = np.random.uniform(0, np.pi*2, p)
gamma = np.random.uniform(0, np.pi*2, p)

The initial state is a uniform superposition of all the states $|q_1,...,q_n\rangle$

In [5]:
init_state_vect = [1 for i in range(2**n_qubits)]
init_state = get_initial_state_instance('CUSTOM')
init_state.init_args(n_qubits, state_vector=init_state_vect)

The initial circuit prepares the initial state

In [6]:
qr = QuantumRegister(n_qubits)
circuit_init = init_state.construct_circuit('circuit', qr)

We define a function `evolve` that takes a Hamiltonian $H$ and an angle $t$ and returns a circuit component made of the unitary matrix $e^{j H t}$

In [7]:
def evolve(hamiltonian, angle, quantum_registers):
    return hamiltonian.evolve(None, angle, 'circuit', 1,
                              quantum_registers=quantum_registers,
                              expansion_mode='suzuki',
                              expansion_order=3)

To create the circuit, we need to compose the different unitary matrice given by `evolve`.

In [8]:
def create_circuit(qr, gamma, beta, p):
    circuit_evolv = reduce(lambda x,y: x+y, [evolve(Hm, beta[i], qr) + evolve(Hc, gamma[i], qr)
                                             for i in range(p)])
    circuit = circuit_init + circuit_evolv
    return circuit

We now create a function `evaluate_circuit` that takes a single vector `gamma_beta` (the concatenation of `gamma` and `beta`) and returns $\langle H_c \rangle = \langle \psi | H_c | \psi \rangle$ where $\psi$ is defined by the circuit created with the function above.

In [9]:
def evaluate_circuit(gamma_beta, qr, p):
    n = len(gamma_beta)//2
    circuit = create_circuit(qr, gamma_beta[:n], gamma_beta[n:], p)
    return np.real(Hc.eval("matrix", circuit, 'statevector_simulator')[0])
evaluate = partial(evaluate_circuit, qr=qr, p=p)

Finally, we optimize the angles:

In [10]:
result = minimize(evaluate, np.concatenate([gamma, beta]), method='L-BFGS-B')
result

      fun: -0.999999999999984
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([2.22044605e-07, 0.00000000e+00, 0.00000000e+00, 4.66293670e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 45
      nit: 7
   status: 0
  success: True
        x: array([5.49778718, 2.84960316, 3.07191941, 3.53429176])

# Analysis of the results

We create a circuit using the optimal parameters found.

In [11]:
circuit = create_circuit(qr, result['x'][:p], result['x'][p:], p)

We use the `statevector_simulator` backend in order to display the state created by the circuit.

In [12]:
backend = Aer.get_backend('statevector_simulator')
job = q_execute(circuit, backend)
state = np.asarray(job.result().get_statevector(circuit))
print(np.absolute(state))
print(np.angle(state))

[7.07106781e-01 4.74641116e-08 4.74641111e-08 7.07106781e-01]
[0.78539819 0.28595117 0.28595115 0.78539819]


We see that the state is approximately $e^{0.79j} \frac{1}{\sqrt{2}} \left( |00 \rangle + |11 \rangle \right)$. It corresponds to a uniform superposition of the two solutions of the classicial problem: $(\sigma_1=1$, $\sigma_2=1)$ and $(\sigma_1=-1$, $\sigma_2=-1)$

Let's now try to evaluate the operators $\sigma^Z_1$ and $\sigma^Z_2$ independently:

In [13]:
Z0 = pauli_z(0, 1)
Z1 = pauli_z(1, 1)

In [14]:
print(Z0.eval("matrix", circuit, "statevector_simulator")[0])
print(Z1.eval("matrix", circuit, "statevector_simulator")[0])

(1.887379141862766e-15+0j)
(1.887379141862766e-15+0j)


We see that both are approximatively equal to zero. It's expected given the state we found above and corresponds a typical quantum behavior where $\mathbb{E}[\sigma^Z_1 \sigma^Z_2] \neq \mathbb{E}[\sigma^Z_1] \mathbb{E}[\sigma^Z_2]$